In [ ]:
Se crea la tabla del dataset final cogiendo la taxonomía del VMR, el listado de PHROGs oficial y los resultados de 
la anotación con Pharokka

In [ ]:
Generar la tabla de taxonomía del VMR:

In [14]:
import pandas as pd

CSV_PATH = "/home/alumno08/TFM/fase2/VMR_MSL40.v1.csv"

# Leer el archivo CSV detectando delimitador automáticamente
vmr_df = pd.read_csv(CSV_PATH, sep=None, engine='python')
vmr_df.columns = vmr_df.columns.str.strip()

# Seleccionar columnas taxonómicas
columnas_tax = ['Virus GENBANK accession', 'Kingdom', 'Phylum', 'Class', 'Family', 'Genus', 'Species']
df_taxonomia = vmr_df[columnas_tax].dropna(subset=['Virus GENBANK accession']).set_index('Virus GENBANK accession')

# Guardar en TSV
df_taxonomia.to_csv("df_taxonomia.tsv", sep='\t')

print("DataFrame de taxonomía creado correctamente y guardado como 'df_taxonomia.tsv'")

DataFrame de taxonomía creado correctamente y guardado como 'df_taxonomia.tsv'


In [ ]:
Crear el listado de PHROGs por genoma, con 0s y 1s:

In [8]:
import os
import re
import shutil

# Rutas
PHAROKKA_OUT = "/home/alumno08/TFM/fase2/pharokka_out"
DESCARTES_DIR = "/home/alumno08/TFM/fase2/descartes"

# Crear carpeta de descartes si no existe
os.makedirs(DESCARTES_DIR, exist_ok=True)

# Expresión regular para nombres válidos (solo letras, números, guiones, guiones bajos y puntos)
patron_valido = re.compile(r"^[\w.-]+$")

# Recorrer todos los elementos del directorio
for nombre in os.listdir(PHAROKKA_OUT):
    ruta = os.path.join(PHAROKKA_OUT, nombre)

    if os.path.isdir(ruta):
        mover = False

        # Caso especial: .ipynb_checkpoints
        if nombre == ".ipynb_checkpoints":
            mover = True
        # Otros nombres con caracteres no válidos
        elif not patron_valido.match(nombre):
            mover = True

        if mover:
            destino = os.path.join(DESCARTES_DIR, nombre)
            print(f"Moviendo {nombre} a descartes/")
            shutil.move(ruta, destino)

Moviendo .ipynb_checkpoints a descartes/


In [9]:
import os
import pandas as pd
from tqdm import tqdm

# Rutas
PHAROKKA_OUT = "/home/alumno08/TFM/fase2/pharokka_out"
OUTFILE = "/home/alumno08/TFM/fase2/df_phrogs_binaria.tsv"

# Leer solo directorios reales (genomas anotados)
genomas_anotados = sorted([
    d for d in os.listdir(PHAROKKA_OUT)
    if os.path.isdir(os.path.join(PHAROKKA_OUT, d))
])

phrog_dict = {}
for genoma_id in tqdm(genomas_anotados, desc="Procesando genomas anotados"):
    phrog_file = os.path.join(PHAROKKA_OUT, genoma_id, "pharokka_cds_final_merged_output.tsv")
    try:
        df_phrog = pd.read_csv(phrog_file, sep="\t")
        if "phrog" not in df_phrog.columns:
            raise ValueError("La columna 'phrog' no está en el archivo.")
        phrogs = df_phrog["phrog"].dropna().unique()
        phrogs = [p for p in phrogs if p != "No_PHROG"]
        phrog_dict[genoma_id] = phrogs
    except Exception as e:
        print(f"Error leyendo PHROGs en {genoma_id}: {e}")
        phrog_dict[genoma_id] = []

# Obtener PHROGs únicos
phrogs_unicos = sorted(set(
    phrog for lista in phrog_dict.values() for phrog in lista if isinstance(phrog, str)
))

# Crear DataFrame binario eficiente
df_bin = pd.DataFrame(0, index=genomas_anotados, columns=[f"PHROG_{p}" for p in phrogs_unicos])
for genoma_id, lista in phrog_dict.items():
    for phrog in lista:
        col = f"PHROG_{phrog}"
        if col in df_bin.columns:
            df_bin.at[genoma_id, col] = 1

# Guardar matriz
df_bin.to_csv(OUTFILE, sep="\t")
print(f"Matriz binaria guardada en: {OUTFILE}")

Procesando genomas anotados:   2%|▏         | 141/6699 [00:00<00:13, 470.53it/s]

Error leyendo PHROGs en AE006468: [Errno 2] No such file or directory: '/home/alumno08/TFM/fase2/pharokka_out/AE006468/pharokka_cds_final_merged_output.tsv'


Procesando genomas anotados: 100%|██████████| 6699/6699 [00:14<00:00, 460.00it/s]


Matriz binaria guardada en: /home/alumno08/TFM/fase2/df_phrogs_binaria.tsv


In [10]:
import shutil
import os

# Rutas
origen = "/home/alumno08/TFM/fase2/pharokka_out/AE006468"
destino = "/home/alumno08/TFM/fase2/descartes/AE006468"

# Crear carpeta descartes si no existe
os.makedirs(os.path.dirname(destino), exist_ok=True)

# Mover el directorio
if os.path.isdir(origen):
    shutil.move(origen, destino)
    print("Directorio AE006468 movido a descartes/")
else:
    print("El directorio AE006468 no existe en pharokka_out.")

Directorio AE006468 movido a descartes/


In [ ]:
He limpiado pharokka_out de genomas con nombres raros y vacíos

In [11]:
import os
import pandas as pd
from tqdm import tqdm

# Rutas
PHAROKKA_OUT = "/home/alumno08/TFM/fase2/pharokka_out"
OUTFILE = "/home/alumno08/TFM/fase2/df_phrogs_binaria.tsv"

# Leer solo directorios reales (genomas anotados)
genomas_anotados = sorted([
    d for d in os.listdir(PHAROKKA_OUT)
    if os.path.isdir(os.path.join(PHAROKKA_OUT, d))
])

phrog_dict = {}
for genoma_id in tqdm(genomas_anotados, desc="Procesando genomas anotados"):
    phrog_file = os.path.join(PHAROKKA_OUT, genoma_id, "pharokka_cds_final_merged_output.tsv")
    try:
        df_phrog = pd.read_csv(phrog_file, sep="\t")
        if "phrog" not in df_phrog.columns:
            raise ValueError("La columna 'phrog' no está en el archivo.")
        phrogs = df_phrog["phrog"].dropna().unique()
        phrogs = [p for p in phrogs if p != "No_PHROG"]
        phrog_dict[genoma_id] = phrogs
    except Exception as e:
        print(f"Error leyendo PHROGs en {genoma_id}: {e}")
        phrog_dict[genoma_id] = []

# Obtener PHROGs únicos
phrogs_unicos = sorted(set(
    phrog for lista in phrog_dict.values() for phrog in lista if isinstance(phrog, str)
))

# Crear DataFrame binario eficiente
df_bin = pd.DataFrame(0, index=genomas_anotados, columns=[f"PHROG_{p}" for p in phrogs_unicos])
for genoma_id, lista in phrog_dict.items():
    for phrog in lista:
        col = f"PHROG_{phrog}"
        if col in df_bin.columns:
            df_bin.at[genoma_id, col] = 1

# Guardar matriz
df_bin.to_csv(OUTFILE, sep="\t")
print(f"Matriz binaria guardada en: {OUTFILE}")

Procesando genomas anotados: 100%|██████████| 6698/6698 [00:14<00:00, 460.44it/s]


Matriz binaria guardada en: /home/alumno08/TFM/fase2/df_phrogs_binaria.tsv


In [ ]:
Ahora vamos a por la info genómica

In [12]:
import os
import pandas as pd
from tqdm import tqdm

# Ruta base de los resultados de Pharokka
PHAROKKA_OUT = "/home/alumno08/TFM/fase2/pharokka_out"
RESUMEN_PATH = "/home/alumno08/TFM/fase2/pharokka_summary_info.tsv"

# Obtener directorios de genomas
genomas = sorted([
    d for d in os.listdir(PHAROKKA_OUT)
    if os.path.isdir(os.path.join(PHAROKKA_OUT, d)) and not d.startswith(".")
])

# Crear lista para recolectar información
info_data = []

for genoma in tqdm(genomas, desc="Extrayendo resumen"):
    resumen_path = os.path.join(PHAROKKA_OUT, genoma, "pharokka_length_gc_cds_density.tsv")
    phrog_path = os.path.join(PHAROKKA_OUT, genoma, "pharokka_cds_final_merged_output.tsv")
    
    try:
        resumen_df = pd.read_csv(resumen_path, sep="\t")
        length = resumen_df["length"].iloc[0]
        gc_perc = resumen_df["gc_perc"].iloc[0]
        
        if os.path.exists(phrog_path):
            genes_df = pd.read_csv(phrog_path, sep="\t")
            num_genes = genes_df.shape[0]
        else:
            num_genes = 0

        info_data.append({
            "genome_id": genoma,
            "genome_length": length,
            "gc_percent": gc_perc,
            "num_genes": num_genes
        })
    except Exception as e:
        print(f"Error procesando {genoma}: {e}")

# Crear DataFrame con la información
df_info = pd.DataFrame(info_data).set_index("genome_id")

# Guardar a archivo TSV
df_info.to_csv(RESUMEN_PATH, sep="\t")
print(f"Resumen guardado en {RESUMEN_PATH}")

Extrayendo resumen: 100%|██████████| 6698/6698 [00:19<00:00, 348.41it/s]


Resumen guardado en /home/alumno08/TFM/fase2/pharokka_summary_info.tsv


In [ ]:
ahora toca combinarlo todo:
- Matriz binaria de PHROGs (df_phrogs_binaria.tsv)

- Taxonomía (df_taxonomia.tsv)

- Información genómica (pharokka_summary_info.tsv)

In [13]:
import pandas as pd

# Cargar matriz binaria de PHROGs
df_phrogs = pd.read_csv(
    "/home/alumno08/TFM/fase2/df_phrogs_binaria.tsv", 
    sep="\t", 
    index_col=0
)

# Cargar taxonomía
df_tax = pd.read_csv(
    "/home/alumno08/TFM/fase2/df_taxonomia.tsv", 
    sep="\t", 
    index_col=0
)

# Cargar información genómica (longitud, GC, nº genes)
df_info = pd.read_csv(
    "/home/alumno08/TFM/fase2/pharokka_summary_info.tsv", 
    sep="\t", 
    index_col=0
)

# Combinar todo
df_total = df_phrogs.join([df_tax, df_info], how="inner")

# Guardar el resultado final
output_path = "/home/alumno08/TFM/fase2/matriz_phrogs_tax_gc.tsv"
df_total.to_csv(output_path, sep="\t")

# Resumen
print(f"Matriz combinada guardada en {output_path}")
print(f"{df_total.shape[0]} genomas y {df_total.shape[1]} columnas en total")

Matriz combinada guardada en /home/alumno08/TFM/fase2/matriz_phrogs_tax_gc.tsv
6692 genomas y 26722 columnas en total


In [14]:
import pandas as pd

# Ruta al archivo consolidado
ruta = "/home/alumno08/TFM/fase2/matriz_phrogs_tax_gc.tsv"

# Cargar el DataFrame
df = pd.read_csv(ruta, sep="\t", index_col=0)

# Mostrar las 20 primeras filas
print(df.head(20))

/tmp/ipykernel_2747161/212801511.py:7: DtypeWarning: Columns (26717) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta, sep="\t", index_col=0)


          PHROG_1  PHROG_10  PHROG_100  PHROG_1000  PHROG_10000  PHROG_10001  \
AB002632        0         0          0           0            0            0   
AB008550        1         0          0           0            0            0   
AB009866        0         1          0           0            0            0   
AB012573        0         0          0           0            0            0   
AB012574        0         0          0           0            0            0   
AB043678        0         0          0           0            0            0   
AB043679        0         0          0           0            0            0   
AB044554        1         1          0           0            0            0   
AB045978        0         1          0           0            0            0   
AB063393        1         0          0           0            0            0   
AB231700        0         0          0           0            0            0   
AB243556        0         1          0  

In [2]:
import pandas as pd

# Cargar matriz actual
ruta_matriz = "/home/alumno08/TFM/fase2/matriz_phrogs_tax_gc.tsv"
df = pd.read_csv(ruta_matriz, sep="\t", index_col=0, low_memory=False)

# Cargar lista oficial de PHROGs
ruta_index = "/home/alumno08/TFM/fase2/PHROG_index.csv"
df_index = pd.read_csv(ruta_index)
todos_phrogs = [p.upper().replace("PHROG_", "PHROG_") for p in df_index["#phrog"].unique()]

# Detectar PHROGs actuales y faltantes
phrogs_actuales = [col for col in df.columns if col.startswith("PHROG_")]
phrogs_faltantes = sorted(set(todos_phrogs) - set(phrogs_actuales), key=lambda x: int(x.split("_")[1]))

# Crear DataFrame de ceros con los PHROGs faltantes
df_faltantes = pd.DataFrame(0, index=df.index, columns=phrogs_faltantes)

# Combinar con el original
df_completo = pd.concat([df, df_faltantes], axis=1)

# Reordenar columnas: primero no PHROGs, luego PHROGs ordenados
otras_cols = [col for col in df.columns if not col.startswith("PHROG_")]
phrog_cols_ordenados = sorted([col for col in df_completo.columns if col.startswith("PHROG_")], key=lambda x: int(x.split("_")[1]))
df_completo = df_completo[otras_cols + phrog_cols_ordenados]

# Guardar
salida = "/home/alumno08/TFM/fase2/matriz_phrogs_tax_gc_final.tsv"
df_completo.to_csv(salida, sep="\t")
print(f"Matriz extendida guardada en: {salida}")

Matriz extendida guardada en: /home/alumno08/TFM/fase2/matriz_phrogs_tax_gc_final.tsv


In [2]:
import pandas as pd

# Ruta al archivo
ruta = "/home/alumno08/TFM/fase2/matriz_phrogs_tax_gc_final.tsv"

# Leer solo las cabeceras para obtener los nombres de columnas
df = pd.read_csv(ruta, sep="\t", nrows=5)

# Mostrar número de columnas
print(f"Número de columnas: {df.shape[1]}")

# Mostrar nombres de las columnas
print("Nombres de las columnas:")
print(df.columns.tolist())

# Leer solo la primera columna para contar filas sin cargar todo el dataset
num_filas = sum(1 for _ in open(ruta)) - 1  # Restamos 1 por la cabecera
print(f"Número de filas (excluyendo cabecera): {num_filas}")

Número de columnas: 38890
Nombres de las columnas:
['Unnamed: 0', 'Kingdom', 'Phylum', 'Class', 'Family', 'Genus', 'Species', 'genome_length', 'gc_percent', 'num_genes', 'PHROG_1', 'PHROG_2', 'PHROG_3', 'PHROG_4', 'PHROG_5', 'PHROG_6', 'PHROG_7', 'PHROG_8', 'PHROG_9', 'PHROG_10', 'PHROG_11', 'PHROG_12', 'PHROG_13', 'PHROG_14', 'PHROG_15', 'PHROG_16', 'PHROG_17', 'PHROG_18', 'PHROG_19', 'PHROG_20', 'PHROG_21', 'PHROG_22', 'PHROG_23', 'PHROG_24', 'PHROG_25', 'PHROG_26', 'PHROG_27', 'PHROG_28', 'PHROG_29', 'PHROG_30', 'PHROG_31', 'PHROG_32', 'PHROG_33', 'PHROG_34', 'PHROG_35', 'PHROG_36', 'PHROG_37', 'PHROG_38', 'PHROG_39', 'PHROG_40', 'PHROG_41', 'PHROG_42', 'PHROG_43', 'PHROG_44', 'PHROG_45', 'PHROG_46', 'PHROG_47', 'PHROG_48', 'PHROG_49', 'PHROG_50', 'PHROG_51', 'PHROG_52', 'PHROG_53', 'PHROG_54', 'PHROG_55', 'PHROG_56', 'PHROG_57', 'PHROG_58', 'PHROG_59', 'PHROG_60', 'PHROG_61', 'PHROG_62', 'PHROG_63', 'PHROG_64', 'PHROG_65', 'PHROG_66', 'PHROG_67', 'PHROG_68', 'PHROG_69', 'PHROG_70',